# Snowballing, for rounds 2 and above

This takes each newly-coded spreadsheet of papers and queries further 'related articles' from the newly selected papers. Each time it runs it retrieves a new set of candidates and adds them to the list, saving them to a new spreadsheet.

Before starting, the previous spreadsheet, PapersToCode.xlsx (starting with N=2), should be double coded, adding coding values (7 or above means 'accept') in the column for each coder (Charles and Pierre) and in the AgreedScore column (the result of coder discussion). Save the result as CodedPapers.xlsx.

Change the coder names in the FirstStep and Analysis notebook if desired.

Copyright &copy; 2021 Charles Weir.   
Shared under the Apache 2.0 License

In [1]:
# We upped this count after round 3 to 19 (see Analysis notebook)
RequiredAnnualCount=8

In [2]:
# !pip install google-search-results
from serpapi import GoogleScholarSearch
from ScholarUtils import GetPapers, GetPaper, WellCitedPapers, InitScholar, RelatedQuery

In [3]:
import pandas as pd
import numpy as np
import pickle
# Reload ScholarUtils every time before executing code 
%load_ext autoreload
%autoreload 2

In [4]:
InitScholar("APIKey.yaml")

In [5]:
scoringSpreadsheet="CodedPapers.xlsx"
previousPapersDf=(pd.read_excel(scoringSpreadsheet, index_col=0,
                                # Want Key as string, not number, to match WellCitedPapers() output:
                              dtype={'Key':np.str_})
                         )
CurrentRound=previousPapersDf.Round.max()
CurrentRound

4

In [6]:
# Get the accepted papers from the latest round of coding:
papersToFollowDf=previousPapersDf.query('Round=={} and AgreedScore > 6'.format(CurrentRound))
print(len(papersToFollowDf))
papersToFollowDf

4


,Key,Round,Citations,Year,Title,Authors,Link,Related,Snippet,AgreedScore,NeedsDiscuss,Pierre,P Comment,Charles,C comment,Agreement,Unnamed: 16
514,662201613823385404,4,133,2018,Do developers update their library dependencies?,"RG Kula, DM German, A Ouni, T Ishio…",https://link.springer.com/article/10.1007/s106...,PB-i-ducMAkJ,Third-party library reuse has become common pr...,8.0,1.0,8.0,8 (3 2 3),8.0,NaN,NaN,NaN
558,8515816864495030621,4,92,2018,Lessons from building static analysis tools at...,"C Sadowski, E Aftandilian, A Eagle…",https://pub-tools-public-publication-data.stor...,XdGBzFFCLnYJ,SOFTWARE BUGS COST developers and software com...,8.0,1.0,8.0,8 (3 3 2),6.0,Is this almost a duplicate?,NaN,NaN
604,3189859254953420294,4,65,2017,DevSecOps: a multivocal literature review,"H Myrbakken, R Colomo-Palacios",https://www.researchgate.net/profile/Havard-My...,Bqb9fheoRCwJ,Involving security in DevOps has been a challe...,7.0,1.0,5.0,5 (2 1 2),8.0,NaN,NaN,NaN
612,5051860876178524246,4,107,2015,A descriptive study of Microsoft's threat mode...,"R Scandariato, K Wuyts, W Joosen",https://link.springer.com/article/10.1007/s007...,VohQc7PQG0YJ,Microsoft's STRIDE is a popular threat modelin...,8.0,1.0,5.0,5 (2 1 2),9.0,NaN,NaN,NaN


In [7]:
# Ask Google for the 'related publications' for each newly coded item (takes a long time)
# Concatenate returned lists...
newPapers = [foundPaper for relatedPaper in papersToFollowDf.itertuples()
                 for foundPaper in GetPapers(RelatedQuery(relatedPaper.Related))
            ] 
len(newPapers)


Retrieving 101 papers for {'q': 'related:PB-i-ducMAkJ:scholar.google.com/'}
Retrieving 101 papers for {'q': 'related:XdGBzFFCLnYJ:scholar.google.com/'}
Retrieving 101 papers for {'q': 'related:Bqb9fheoRCwJ:scholar.google.com/'}
Retrieving 101 papers for {'q': 'related:VohQc7PQG0YJ:scholar.google.com/'}


404

In [8]:
with open( "papersFound.p", "rb" ) as inFile:
    allPapers=pickle.load( inFile ) + newPapers

In [9]:
# The Google queries take a long time and can cost money. 
# We save the results as 'pickle' dumps, 
# Can also 'comment out' the slow parts by changing the cell type to 'Raw NB Convert'; 
# to redo the queries, convert them back again to Code.

with open( "papersFound.p", "wb" ) as outFile:
    pickle.dump( allPapers, outFile )

In [10]:
# How many unique papers so far?

pd.Series([paper['result_id'] for paper in allPapers]).unique().size

4238

In [11]:
newPapersDf=(WellCitedPapers(newPapers, requiredAnnualCount=RequiredAnnualCount)
           .reindex(columns=previousPapersDf.columns, fill_value='') # Add in the extra classification columns.
            .assign(Round=CurrentRound+1)
          )
len(newPapersDf)

61

In [12]:
allPapersSoFarDf=(pd.concat([previousPapersDf, newPapersDf]) # Keep already coded papers - don't want to recode.
                .drop_duplicates(subset=['Key'])  # Only want new ones - this keeps the earlier entries.
                .reset_index(drop=True)           # Don't need to keep the old numbering.
               )
allPapersSoFarDf.to_excel('PapersToCode.xlsx')
len(allPapersSoFarDf)
#allPapersSoFarDf

645

In [13]:
!open PapersToCode.xlsx

In [14]:
assert(False)

AssertionError: 

In [ ]:
!open ScholarUtils.py